In [4]:
import pandas as pd
import ast
import numpy as np

print("Cargando tu dataset unido...")
try:
    # Cargamos el archivo que guardamos en el paso anterior
    df = pd.read_csv('movies_dataset_unido.csv')
    print(f"¡Dataset cargado! Forma: {df.shape}")
except FileNotFoundError:
    print("Error: No se encontró 'movies_dataset_unido.csv'.")
    # Manejar el error si es necesario

Cargando tu dataset unido...


ParserError: Error tokenizing data. C error: EOF inside string starting at row 3926

In [2]:
print("Limpiando valores nulos...")

# 1. Rellenar 'overview'
df['overview'] = df['overview'].fillna('')

# 2. Rellenar las columnas estructurales
df['genres'] = df['genres'].fillna('[]')
df['keywords'] = df['keywords'].fillna('[]')
df['cast'] = df['cast'].fillna('[]')
df['crew'] = df['crew'].fillna('[]')

print("Valores nulos limpiados.")

Limpiando valores nulos...


NameError: name 'df' is not defined

In [ ]:
# --- Función 1: El evaluador seguro ---
# Esta función convierte el string '[]' en un objeto lista []
def safe_literal_eval(s):
    try:
        # Intenta convertir el string en un objeto Python (ej. lista)
        return ast.literal_eval(s)
    except (ValueError, SyntaxError, TypeError):
        # Si falla (ej. es un NaN malformado), devuelve una lista vacía
        return []

# --- Función 2: Extraer Nombres (para Genres y Keywords) ---
def get_names(data_list_str):
    # 1. Convertir el string (ej. "[{'name': 'Action'}]") en un objeto lista
    data_list = safe_literal_eval(data_list_str)

    if isinstance(data_list, list):
        names = []
        for item in data_list:
            if 'name' in item:
                # 2. Estandarizar: minúsculas y sin espacios
                clean_name = item['name'].lower().replace(" ", "")
                names.append(clean_name)
        return names
    return []

# --- Función 3: Extraer Top 3 Actores (para Cast) ---
def get_top_cast(cast_list_str, top_n=3):
    cast_list = safe_literal_eval(cast_list_str)

    if isinstance(cast_list, list):
        names = []
        # 3. Limitar a los primeros 'top_n' (3)
        for item in cast_list[:top_n]:
            if 'name' in item:
                # 2. Estandarizar
                clean_name = item['name'].lower().replace(" ", "")
                names.append(clean_name)
        return names
    return []

# --- Función 4: Extraer al Director (para Crew) ---
def get_director(crew_list_str):
    crew_list = safe_literal_eval(crew_list_str)

    if isinstance(crew_list, list):
        # 4. Buscar en toda la lista
        for member in crew_list:
            if member.get('job') == 'Director':
                # 2. Estandarizar
                clean_name = member['name'].lower().replace(" ", "")
                return clean_name
    # Si no se encuentra director o la lista está vacía
    return ""

print("Funciones de 'parsing' creadas exitosamente.")

Funciones de 'parsing' creadas exitosamente.


In [ ]:
print("Aplicando funciones al DataFrame... (Esto puede tardar un momento)")

# 'axis=1' no es necesario aquí, .apply() funciona sobre la Serie
df['genres_list'] = df['genres'].apply(get_names)
df['keywords_list'] = df['keywords'].apply(get_names)
df['cast_list'] = df['cast'].apply(get_top_cast)
df['director'] = df['crew'].apply(get_director) # Esta será un string

print("¡Nuevas columnas creadas!")

# Verifiquemos el resultado
print("\n--- Ejemplo de limpieza (primera película) ---")
print(f"Título: {df.iloc[0]['title']}")
print(f"Géneros Limpios: {df.iloc[0]['genres_list']}")
print(f"Actores Limpios: {df.iloc[0]['cast_list']}")
print(f"Director Limpio: {df.iloc[0]['director']}")

Aplicando funciones al DataFrame... (Esto puede tardar un momento)
¡Nuevas columnas creadas!

--- Ejemplo de limpieza (primera película) ---
Título: Toy Story
Géneros Limpios: ['animation', 'comedy', 'family']
Actores Limpios: ['tomhanks', 'timallen', 'donrickles']
Director Limpio: johnlasseter


In [ ]:
print("Forzando la limpieza de 'overview' y 'director'...")

# 1. Asegura que 'overview' sea string (corrige el error anterior)
df['overview'] = df['overview'].fillna('')

# 2. Asegura que 'director' sea string (corrige TU NUEVO ERROR)
# Reemplaza cualquier NaN (float) por un string vacío
df['director'] = df['director'].fillna('')

print("¡Columnas limpiadas y listas!")

Forzando la limpieza de 'overview' y 'director'...
¡Columnas limpiadas y listas!


PREPARACION DEL ENTRONO


In [ ]:
def create_metadata_soup(row):
    # Convertir las listas en strings separados por espacios
    genres = ' '.join(row['genres_list'])
    keywords = ' '.join(row['keywords_list'])
    cast = ' '.join(row['cast_list'])

    # Aplicar la ponderación (repetición)
    # (El espacio extra ' ' es clave para que no se peguen las palabras)
    genres_weighted = (genres + ' ') * 2
    keywords_weighted = (keywords + ' ') * 3
    cast_weighted = (cast + ' ') * 3
    director_weighted = (row['director'] + ' ') * 3

    # Unir todo
    return (
        row['overview'] + ' ' +
        genres_weighted +
        keywords_weighted +
        cast_weighted +
        director_weighted
    )

print("\nCreando la 'metadata_soup'...")
# Usamos axis=1 para que la función pueda acceder a las columnas de cada fila
df['metadata_soup'] = df.apply(create_metadata_soup, axis=1)

print("¡'metadata_soup' creada!")

# --- Verificación Final ---
print("\n--- Ejemplo de Sopa (primera película) ---")
print(f"Título: {df.iloc[0]['title']}")
print(f"Sopa (primeros 500 caracteres):\n{df.iloc[0]['metadata_soup'][:500]}")


Creando la 'metadata_soup'...
¡'metadata_soup' creada!

--- Ejemplo de Sopa (primera película) ---
Título: Toy Story
Sopa (primeros 500 caracteres):
Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences. animation comedy family animation comedy family jealousy toy boy friendship friends rivalry boynextdoor newtoy toycomestolife jealousy toy boy friendship friends rivalry boynextdoor newtoy toycome
